In [1]:
import numpy as np
import mxnet as mx
from mxnet import nd
from tqdm import tqdm

# Image Preprocess 

- size: > 224 
- normalized: mean = [0.485, 0.456, 0.406],  std = [0.229, 0.224, 0.225]
- shape:  RGB images of shape (N x 3 x H x W)

https://mxnet.incubator.apache.org/api/python/gluon/model_zoo.html#mxnet.gluon.model_zoo

In [2]:
preprocess_list = [
    lambda img: img.astype("float32")/255,
    mx.image.ForceResizeAug((224, 224)),
    mx.image.ColorNormalizeAug(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),
    lambda img: nd.transpose(img,(2,0,1))
]

def image_preprocess(img):
    for f in preprocess_list:
        img = f(img)
    return img

# Data Loader

In [3]:
def transform(img, label):
    return image_preprocess(img), label

def load_data(data_dir, load_batch_size = 32, f_trans=transform):
    imgs = mx.gluon.data.vision.ImageFolderDataset(data_dir, transform=transform)
    data = mx.gluon.data.DataLoader(imgs, load_batch_size, last_batch="keep")
    return data

In [4]:
train_data = load_data("/home/guangyaoyang/data/dogBreed/train_gy")
test_data = load_data("/home/guangyaoyang/data/dogBreed/test_gy")

# Pretrained Net Loader

In [5]:
ctx = mx.gpu()
resnet50_v2 = mx.gluon.model_zoo.vision.resnet50_v2(pretrained=True, ctx=ctx)

# Feature Extraction From Pretrained Net

In [6]:
def extract_features(net, data, ctx):
    rst_X, rst_y = [], []
    for X, y in tqdm(data):
        Xi = net.features(X.as_in_context(ctx))
        rst_X.append(Xi.asnumpy())
        rst_y.append(y.asnumpy())
    return np.concatenate(rst_X, axis=0), np.concatenate(rst_y, axis=0) 

In [7]:
X_train_resnet50_v2, y_train = extract_features(resnet50_v2, train_data, ctx)

100%|██████████| 320/320 [02:39<00:00,  1.18it/s]


In [9]:
X_test_resnet50_v2, _ = extract_features(resnet50_v2, test_data, ctx)

100%|██████████| 324/324 [01:36<00:00,  1.37it/s]


# Save features and labels for future use

In [10]:
import h5py

with h5py.File('/home/guangyaoyang/data/dogBreed/resnet50_v2_pretrained_Xy.h5', 'w') as f:
    f['X_train_resnet50_v2'] = X_train_resnet50_v2
    f['X_test_resnet50_v2'] = X_test_resnet50_v2
    f['y_train'] = y_train